In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping
from data_generation import DenseGenerator, ChessPositionGen

import datetime
%load_ext tensorboard

In [2]:
# Setting paramaters on early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          mode='min',
                          restore_best_weights=True)

log_dir = "logs/fit/dense" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [3]:
# Memory management, likely not necessary, but used as a safety as per the documentation recommendations on using GPUS

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [4]:
train = pd.read_csv('fens/train.csv')
val = pd.read_csv('fens/val.csv')

In [5]:
train_gen = DenseGenerator(train, batch_size=1024)
val_gen = DenseGenerator(val, batch_size=1024)

## Multi Layer Perceptron model for comparison with the CNN versions

This model uses basic densely connected layers and uses a slight variation of the previously used data generator with no reshaping of the array.

In [6]:
dense_model = models.Sequential()
dense_model.add(layers.Dense(832, input_shape=(832,), activation='relu'))
dense_model.add(layers.Dense(64, activation='relu'))
dense_model.add(layers.Dense(64, activation='relu'))
dense_model.add(layers.Dense(1, activation='sigmoid'))
dense_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc'])
dense_model.summary()

# Fitting the model
dense_history = dense_model.fit(x=train_gen,
                    validation_data=val_gen,
                    # steps_per_epoch=100,
                    epochs=15,
                    callbacks=[earlystop, tensorboard_callback]
                    )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 832)               693056    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                53312     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 750,593
Trainable params: 750,593
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
1977/1977 [==============================] - 2590s 1s/step - loss: 0.4226 - acc: 0.8320 - val_loss: 0.3911 - val_acc: 0.8487
Epoch 2/15
1977/1977 [==============================] - 2598s 1s/step - loss: 0.3766 - acc: 0.8544 - val_loss: 0

In [9]:
dense_model.save('MLPmodel-Long-PB')

INFO:tensorflow:Assets written to: DenseModel-PB/assets


In [10]:
dense_history = dense_model.fit(x=train_gen,
                    validation_data=val_gen,
                    epochs=15,
                    callbacks=[earlystop, tensorboard_callback])

Epoch 1/15
1977/1977 [==============================] - 2595s 1s/step - loss: 0.2872 - acc: 0.8874 - val_loss: 0.3181 - val_acc: 0.8767
Epoch 2/15
1977/1977 [==============================] - 2590s 1s/step - loss: 0.2852 - acc: 0.8881 - val_loss: 0.3158 - val_acc: 0.8776ETA: 2:49 - l - ETA: 22s
Epoch 3/15
1977/1977 [==============================] - 2582s 1s/step - loss: 0.2833 - acc: 0.8888 - val_loss: 0.3150 - val_acc: 0.8778
Epoch 4/15
1977/1977 [==============================] - 2581s 1s/step - loss: 0.2816 - acc: 0.8895 - val_loss: 0.3151 - val_acc: 0.8781
Epoch 5/15
1977/1977 [==============================] - 2569s 1s/step - loss: 0.2800 - acc: 0.8899 - val_loss: 0.3153 - val_acc: 0.8774
Epoch 6/15
1977/1977 [==============================] - 2583s 1s/step - loss: 0.2784 - acc: 0.8904 - val_loss: 0.3144 - val_acc: 0.8785
Epoch 7/15
1977/1977 [==============================] - 2601s 1s/step - loss: 0.2770 - acc: 0.8909 - val_loss: 0.3159 - val_acc: 0.8780
Epoch 8/15
1977/1977 [==

In [12]:
dense_model.save('MLPmodel-Long.h5')

## Autotuning